# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

To have the "source" only add new smoke at the first frame, set `constant_source = False` (found in cell 2). To have the "source" constantly add new smoke, set it to `True`

In [1]:
import taichi as ti
import taichi.math as tm

ti.init()

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, osx, python 3.9.7
[I 11/07/22 14:16:34.901 298407] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64


In [9]:
### Global Variables ###

substeps = 5
iters = 20
dt = 0.11 
res = 500
dx = 0.015

vel = ti.Vector.field(2, float, shape=(res, res))
vel_copy = ti.Vector.field(2, float, shape=(res,res))
vel_divs = ti.field(float, shape=(res, res))
pressure_hat = ti.field(float, shape=(res, res))
new_pressure = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
den_copy = ti.field(float, shape=(res,res))
vor = ti.field(float, shape=(res,res))
N = ti.Vector.field(2, float, shape=(res, res))

new_src_vel = 3.0
src_pos = tm.vec2(0.5, 0.5)
src_vel = tm.vec2(3, 0)
src_rad = 0.1

colors = ti.Vector.field(3, float, shape=(res,res))

debug = True

# tester variables
smoke_check = ti.field(float, shape=())
vel_check = ti.Vector.field(2, float, shape=())
vel_check_surrounding = ti.Vector.field(2, float, shape=5)
new_pos_check = ti.Vector.field(2, float, shape=())
vor_check = ti.field(float, shape=())
f_N_check = ti.Vector.field(3, float, shape=())
f_N_2D_check = ti.Vector.field(2, float, shape=())
pressure_check = ti.field(float, shape=())
iter_pressure_checks = ti.field(float, shape=iters)

ctr = ti.field(int, shape=())

In [30]:
### Helper Functions ###

# @param - coordinates (i,j) of the node
# @return - gradient of p at node (i,j)
@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    pressures = tm.vec2(pressure_hat[i+1, j] - pressure_hat[i-1, j], pressure_hat[i, j+1] - pressure_hat[i, j-1])
    return pressures / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - divergence of the velocity at grid node (i, j)
@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the curl of the velocity at grid node (i, j)
@ti.func
def curl_vel(i:ti.i32, j:ti.i32):
#     if debug and i == 250 and j == 210:
#         vel_check_surrounding[0] = vel[i+1, j]
#         vel_check_surrounding[1] = vel[i-1, j]
#         vel_check_surrounding[2] = vel[i, j+1]
#         vel_check_surrounding[3] = vel[i, j-1]
    ctr[None] += 1
    return 1.0
#     return (vel[i+1, j][1] - vel[i-1, j][1] - vel[i, j+1][0] + vel[i, j-1][0]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the laplacian of the pressure at grid node (i, j)
@ti.func
def laplacian_pressure(i:ti.i32, j:ti.i32):
    return (pressure_hat[i+1, j] + pressure_hat[i-1, j] + pressure_hat[i, j+1] + pressure_hat[i, j-1] - 4 * pressure_hat[i, j]) / (dx * dx)

# returns whether or not (row, col) is on a boundary
@ti.func
def isBoundary(row:ti.i32, col:ti.i32) -> bool:
    return (row == 0 or col == 0 or row == res-1 or col == res-1)

In [31]:
### Simulator Functions ###

# @param - x_p: vec2, where x_p = (x - u^(n+1/2)(x) * dt) (the "starting point")
# @return - the interpolated value for u*(x) at x_p
@ti.func
def interpolate(x_p, vector_field):
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vector_field[x1, y1] + (x - x1)/(x2 - x1) * vector_field[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vector_field[x1, y2] + (x - x1)/(x2 - x1) * vector_field[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# Fills vel_copy and den_copy
@ti.func
def fill_copies():
    for i in ti.grouped(vel):
        vel_copy[i] = vel[i]
        den_copy[i] = smoke_density[i]

# Advection using a semi-Lagrangian method
@ti.kernel
def advect():
    fill_copies()
    for row in range(res):
        for col in range(res):
            pos = tm.vec2(row, col)
            new_pos = pos - vel[row, col] * 0.5 * dt
            new_vel = interpolate(new_pos, vel_copy)
            new_pos = pos - new_vel * dt
            new_vel = interpolate(new_pos, vel_copy)
            vel[row, col] = new_vel
            
            new_den = interpolate(new_pos, den_copy)
            smoke_density[row, col] = new_den
            
            if debug and row == 250 and col == 200:
                new_pos_check[None] = new_pos
                vel_check[None] = vel[row, col]
                smoke_check[None] = new_den
                vel_check_surrounding[0] = vel[row+1, col]
                vel_check_surrounding[1] = vel[row-1, col]
                vel_check_surrounding[2] = vel[row, col+1]
                vel_check_surrounding[3] = vel[row, col-1]
                vel_check_surrounding[4] = vel[row, col]
    
@ti.func
def swap_pressures():
    for i in ti.grouped(pressure_hat):
        pressure_hat[i] = new_pressure[i]
        
@ti.func
def calc_divs():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)

@ti.func
def gauss_seidel():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                coef_off_dia = 1.0 / (dx * dx)
                coef_dia = 4.0 / (dx * dx)
                off_diagonal = 0.
                off_diagonal += pressure_hat[row-1, col] + pressure_hat[row+1, col] + pressure_hat[row, col+1] + pressure_hat[row, col-1]
#                 new_pressure[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
                pressure_hat[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
        
@ti.func
def correct_vel():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                grad_p = gradient_pressure(row, col)
                vel[row, col] -= grad_p
            
                if debug and row == 250 and col == 200:
                    pressure_check[None] = pressure_hat[row, col]
    
@ti.kernel
def project():
    # Calc velocity for RHS of Poissan (div u*)
    calc_divs
    for i in range(iters):
        gauss_seidel()
        iter_pressure_checks[i] = pressure_hat[250, 250]
#         swap_pressures()
    correct_vel()

@ti.func
def update_vorticity_field():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vor[row, col] = curl_vel(row, col)

            if debug and row == 250 and col == 210:
#                     vor_check[None] = vor[row, col]
                vor_check[None] = curl_vel(row, col)
#                     i = row
#                     j = col
#                     vel_check_surrounding[0] = vel[i+1, j]
#                     vel_check_surrounding[1] = vel[i-1, j]
#                     vel_check_surrounding[2] = vel[i, j+1]
#                     vel_check_surrounding[3] = vel[i, j-1]


@ti.func
def update_N():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                N[row, col][0] = (abs(vor[row+1, col]) - abs(vor[row-1, col]))/ (2 * dx)
                N[row, col][1] = (abs(vor[row, col+1]) - abs(vor[row, col-1]))/ (2 * dx)
                N[row, col] /= (N[row, col].norm() + 10e-10)

@ti.func
def update_velocity():
    vor_conf_coef = 8.0
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                f = vor_conf_coef * dx * tm.cross(tm.vec3(N[row, col][0], N[row, col][1], 0), tm.vec3(0,0,vor[row, col])) # How does this work? N is a 2d vector, and vor is a scalar
                if debug and row == 250 and col == 200:
                    f_N_check[None] = f
                f_2d = tm.vec2(f[0], f[1])
                vel[row, col] += f_2d * dt

@ti.kernel
def vorticity_confinement():
    vor.fill(0.)
    update_vorticity_field()
    update_N() # update N (direction of local rotation)
    update_velocity() # find vorticity confinement force and update velocity

# create a "source" for the smoke
@ti.kernel
def source():
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - src_pos).norm() < src_rad:
                vel[row, col] = src_vel
                smoke_density[row, col] = 1

# 0 = up, 1 = left, 2 = down, 3 = right
@ti.kernel
def directed_source(x:ti.f32, y:ti.f32, direction:ti.i32):
    new_source_pos = tm.vec2(x, y)
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - new_source_pos).norm() < src_rad:
                if direction == 0: # up
                    vel[row, col] = tm.vec2(0, new_src_vel)
                elif direction == 1: # left
                    vel[row, col] = tm.vec2(-new_src_vel, 0)
                elif direction == 2: # down
                    vel[row, col] = tm.vec2(0, -new_src_vel)
                else: # right
                    vel[row, col] = tm.vec2(new_src_vel, 0)
                smoke_density[row, col] = 1

@ti.kernel
def update_colors():
    for i in ti.grouped(colors):
        curr_den = smoke_density[i]
        colors[i] = tm.vec3(curr_den, curr_den, curr_den)

In [32]:
def substep():
    # renew source for each substep if user asks for it
    if constant_source:
        source()
    advect()
#     vorticity_confinement()
#     project()
    update_colors()

In [33]:
### Setup ###
@ti.kernel
def init_grid():
    for i in ti.grouped(vel):
        vel[i] = tm.vec2(0,0)
        N[i] = tm.vec2(0,0)
    for i in ti.grouped(smoke_density):
        smoke_density[i] = 0
        vel_divs[i] = 0
        pressure_hat[i] = 0
        new_pressure[i] = 0
        vor[i] = 0
    ctr[None] = 0

In [34]:
### Function to get print checks ###
@ti.kernel
def get_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    smoke_check[None] = smoke_density[row, col]
    vel_check[None] = vel[row, col]
    vor_check[None] = vor[row, col]
    f_N_2D_check[None] = N[row, col]
    pressure_check[None] = pressure_hat[row, col]

def print_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    print("(row, col) = (%d, %d)" % (row, col))
    print("smoke check", smoke_check[None])
    print("smoke density at middle", smoke_density[250, 250])
    print("vel check", vel_check[None])
    print("vor check", vor_check[None])
    print("N check", f_N_check[None])
    print("pressure check", pressure_check[None])
    print("iter pressure checks", iter_pressure_checks)

In [35]:
def main():
    # gui = ti.GUI('Grid Fluid', (res, res))
    gui = ti.GUI('Grid Fluid', (res, res), fast_gui=True)
    
    global constant_source
    constant_source = False
        
    init_grid()
    if not constant_source:
        source()

    if debug:
        for step in range(substeps):
            print("step ", step)
            substep()
            print("smoke check", smoke_check[None])
            print("smoke density at middle", smoke_density[250, 250])
            print("vel check", vel_check[None])
            print("new pos check", new_pos_check[None])
            print("f_N_check", f_N_check[None])
            print("vor check", vor_check[None])
            print("surrounding velocities", vel_check_surrounding)
            print("pressure check", pressure_check[None])
            print("iter pressure checks", iter_pressure_checks)
            
            print("counter", ctr[None])
        
    while gui.running:
        # Move stuff
        if not debug:
            for step in range(substeps):
                substep()
        
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.SPACE:
                constant_source = not constant_source
            elif e.key == 'w':
                directed_source(e.pos[0], e.pos[1], 0)
            elif e.key == 'a':
                directed_source(e.pos[0], e.pos[1], 1)
            elif e.key == 's':
                directed_source(e.pos[0], e.pos[1], 2)
            elif e.key == 'd':
                directed_source(e.pos[0], e.pos[1], 3)
            elif e.key == ti.GUI.LMB:
                get_checks(e.pos[0], e.pos[1])
                print_checks(e.pos[0], e.pos[1])
             
        # Draw grid
        gui.set_image(colors)
        gui.show()
        
    
if __name__ == '__main__':
    main()

step  0
smoke check 0.7244415283203125
smoke density at middle 1.0
vel check [2.17332458 0.        ]
new pos check [249.72444153 200.        ]
f_N_check [0. 0. 0.]
vor check 0.0
surrounding velocities [[0.        0.       ]
 [0.        0.       ]
 [3.        0.       ]
 [0.        0.       ]
 [2.1733246 0.       ]]
pressure check 0.0
iter pressure checks [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
counter 0
step  1
smoke check 0.5719501972198486
smoke density at middle 1.0
vel check [1.71585059 0.        ]
new pos check [249.789505 200.      ]
f_N_check [0. 0. 0.]
vor check 0.0
surrounding velocities [[0.        0.       ]
 [0.        0.       ]
 [3.        0.       ]
 [0.        0.       ]
 [1.7158506 0.       ]]
pressure check 0.0
iter pressure checks [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
counter 0
step  2
smoke check 0.47418734431266785
smoke density at middle 1.0
vel check [1.422562 0.      ]
new pos check [249.82907104 200.        ]
f_N_ch

## TODO

Check that the pressures are correct

Print out the pressure_hat at the end of each iteration to see that it is approaching the correct solution

Draw velocity field to see if it is divergence free\
Or draw the color for the velocity field to visualize

lap(p) - RHS

Change to use fewer for loops

check vor and N